# Дипломное задание по профессии
# «Data Scientist»

### это 3 из 3 файлов дипломного задания

## Задание со звёздочкой:

Вы исследовали данные компании!

Но у инфраструктурного отдела произошли неполадки, возникло подозрение о сбое работы логики записи состоявшихся поездок.

Вам доверена задача проверить наличие ошибок в имеющихся данных, чтобы удостовериться в корректности сделанных выводов и помочь коллегам.

# 11. Загрузите файл dip_db_check_taxi.csv (содержит случайный семпл данных заказанных поездок) в pandas dataframe;

проверьте предположение компании о нарушении работы базы данных (фантомном задвоении фиксации заказов такси), для этого постройте коллаборативную фильтрацию по идентификаторам поездок для определения наиболее похожих/идентичных записей, после, удалите дубликаты получившихся пар и сделайте выводы о наличии задвоений и корректности работы базы.

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_theme (style = 'darkgrid')

●	order_gk — идентификатор поездки;

●	weekday_key — день недели совершения поездки;

●	hour_key — час совершения поездки;

●	distance_km — дистанция поездки;

●	offer_class_group — класс поездки (1 - Economy, 2 - Comfort, 3 - Premium).

In [2]:
# !wget 'https://u.netology.ru/backend/uploads/lms/attachments/files/data/52036/dip_db_check_taxi.csv' -O dip_db_check_taxi.csv

In [3]:
df = pd.read_csv('dip_db_check_taxi.csv')
df

,order_gk,weekday_key,hour_key,distance_km,offer_class_group
0,4957,3,27,14.239,1
1,9314,3,21,39.518,2
2,13922,2,21,17.203,2
3,37826,5,19,36.971,2
4,38936,2,19,49.634,2
...,...,...,...,...,...
95,814413,4,22,11.877,3
96,838313,5,24,49.928,1
97,842245,2,15,10.893,3
98,848459,4,20,29.096,3


In [4]:
# Посмотреть пропуски

def prop(data = df, p = 1):

    pr = pd.DataFrame([])
    col = []
    pp = []
    pc = []
    tp = []

    for c in data:
        col.append(c)
        pp.append(data[c].isna().sum())
        pc.append(round(100 * data[c].isna().sum() / len(data), 2))
        tp.append(data.dtypes[c])

    pr[f'Entries: {len(data)}'] = col
    pr['SUM_Null'] = pp
    pr['%_Null'] = pc
    pr['Dtype'] = tp

    if p == 0:
        return pr

    return pr[pr['SUM_Null'] != 0]

In [5]:
prop(df, 0)

,Entries: 100,SUM_Null,%_Null,Dtype
0,order_gk,0,0.0,int64
1,weekday_key,0,0.0,int64
2,hour_key,0,0.0,int64
3,distance_km,0,0.0,float64
4,offer_class_group,0,0.0,int64


In [6]:
# Посмотреть информацию по всем значениям

def data(data = df):

    print('-------------------------------------\nКоличество данных -', len(data))
    print('-------------------------------------\nКоличество дубликатов -', data.duplicated().sum())

    for d in data:
        print('\n-------------------------------------\n', 'Столбец -', d, '||', 
              'Кол-во уникальных значений -', len(data[d].unique()), '||',  
              'Кол-во дубликатов -', data[d].duplicated().sum())
        print('\nУникальные значения:')
        print(np.sort(data[d].unique()))

In [7]:
data(df)

-------------------------------------
Количество данных - 100
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - order_gk || Кол-во уникальных значений - 100 || Кол-во дубликатов - 0

Уникальные значения:
[  4957   9314  13922  37826  38936  43032  61028  68743  68812  86889
  94794 100864 103837 111907 111962 127048 131111 142869 148753 162009
 165000 179226 180162 183685 190596 191612 191880 194882 197454 200303
 204791 218884 226751 229198 243224 261386 273205 273244 274427 289911
 293849 322696 323099 330292 336264 345050 381549 396090 402616 412671
 420234 425322 445704 462142 490311 507071 507841 508570 512649 514421
 516190 537235 544379 549348 554260 558065 588093 593742 594666 597863
 607437 620982 626856 635251 635571 639770 642424 659570 665277 674519
 693398 697044 699904 735232 740380 744471 747749 755209 757162 761044
 792785 796337 797117 804604 805930 814413 838313 842245 848459 867363]

--------------------

---

Как мы видим, пропуском нет.

Представленный дата фрейм содержит 100 записей о совершенных поездках.

Дубликатов по строчкам нет, также дубликаты отсутствуют в столбцах order_gk и distance_km.

Столбец - hour_key содержит ошибочные данные, т.к. в нем указаны ошибочные часы поездки с 24 по 31.

Можно предположить, что это и есть те задвоения, о которых говориться в задании.

Хотя столбец distance_km содержит 100 абсолютно разных значений, соответственно посмотрим близкие к друг другу данные.

---

In [8]:
df[df['hour_key'] > 23]

,order_gk,weekday_key,hour_key,distance_km,offer_class_group
0,4957,3,27,14.239,1
9,86889,2,24,9.565,3
13,111907,3,25,29.400,1
22,180162,5,24,36.224,2
25,191612,4,27,0.052,3
27,194882,3,31,30.770,1
29,200303,1,24,47.530,1
30,204791,4,25,7.643,3
34,243224,1,26,43.644,2
45,345050,7,30,28.108,3


---

Т.к. наше предположение строится на ошибке в часе поездки, то удалим столбцы order_gk и hour_key.

Близость данных будем смотреть без них.

---

In [9]:
df1 = df[df['hour_key'] > 23].copy()
df1.drop(['order_gk', 'hour_key'], axis= 1 , inplace= True )
df1

,weekday_key,distance_km,offer_class_group
0,3,14.239,1
9,2,9.565,3
13,3,29.400,1
22,5,36.224,2
25,4,0.052,3
27,3,30.770,1
29,1,47.530,1
30,4,7.643,3
34,1,43.644,2
45,7,28.108,3


In [10]:
df2 = df[df['hour_key'] <= 23].copy()
df2.drop(['order_gk', 'hour_key'], axis= 1 , inplace= True )
df2

,weekday_key,distance_km,offer_class_group
1,3,39.518,2
2,2,17.203,2
3,5,36.971,2
4,2,49.634,2
5,5,51.285,1
...,...,...,...
91,2,43.235,1
94,1,32.223,3
95,4,11.877,3
97,2,10.893,3


In [11]:
data(df1)

-------------------------------------
Количество данных - 32
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - weekday_key || Кол-во уникальных значений - 7 || Кол-во дубликатов - 25

Уникальные значения:
[1 2 3 4 5 6 7]

-------------------------------------
 Столбец - distance_km || Кол-во уникальных значений - 32 || Кол-во дубликатов - 0

Уникальные значения:
[5.2000e-02 5.9140e+00 7.6430e+00 8.5650e+00 9.5650e+00 1.1004e+01
 1.1013e+01 1.1356e+01 1.2737e+01 1.3323e+01 1.4239e+01 2.1366e+01
 2.4112e+01 2.8108e+01 2.8328e+01 2.8673e+01 2.8692e+01 2.9400e+01
 3.0770e+01 3.1736e+01 3.2750e+01 3.2811e+01 3.5470e+01 3.6224e+01
 3.7879e+01 4.3644e+01 4.4980e+01 4.7530e+01 4.9928e+01 5.6033e+01
 6.6715e+01 8.1507e+01]

-------------------------------------
 Столбец - offer_class_group || Кол-во уникальных значений - 3 || Кол-во дубликатов - 29

Уникальные значения:
[1 2 3]


In [12]:
data(df2)

-------------------------------------
Количество данных - 68
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - weekday_key || Кол-во уникальных значений - 7 || Кол-во дубликатов - 61

Уникальные значения:
[1 2 3 4 5 6 7]

-------------------------------------
 Столбец - distance_km || Кол-во уникальных значений - 68 || Кол-во дубликатов - 0

Уникальные значения:
[ 1.243  1.625  2.255  2.635  3.945  4.208  5.33   6.503  6.668 10.148
 10.893 11.016 11.165 11.348 11.627 11.877 12.378 12.881 16.219 17.203
 18.907 19.193 19.316 20.217 22.122 22.552 22.563 22.6   22.614 24.
 24.445 24.846 25.977 26.371 27.104 28.28  28.594 29.096 29.536 30.6
 30.855 30.881 32.223 32.284 34.158 35.02  36.971 39.518 40.913 43.203
 43.235 45.018 49.22  49.427 49.634 51.285 53.07  53.658 54.047 57.846
 57.879 59.166 60.169 60.397 75.593 83.107 87.552 90.751]

-------------------------------------
 Столбец - offer_class_group || Кол-во уникальных зна

---

Близость данных будем смотреть на косинусной близости векторов.

Сначала посмотрим близость всех данных с часами от 24 до 31 к данным с корректно указанными часами.

---

In [13]:
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
result = pd.DataFrame(columns=['df1', 'blz', 'df2'])

for ind1 in tqdm(df1.index):
    for ind2 in df2.index:
        blz = cosine_similarity (df1.loc[[ind1]], df2.loc[[ind2]])
        res = pd.DataFrame({'df1': [ind1],
                            'blz': [blz[0][0]],
                            'df2': [ind2]})
        result = pd.concat([result, res])
                            
result

  0%|          | 0/32 [00:00<?, ?it/s]

,df1,blz,df2
0,0,0.991182,1
0,0,0.994742,2
0,0,0.997217,3
0,0,0.985667,4
0,0,0.992710,5
...,...,...,...
0,99,0.994316,91
0,99,0.998211,94
0,99,0.965634,95
0,99,0.985379,97


In [15]:
result.sort_values('blz', ascending = False).head(22)

,df1,blz,df2
0,61,1.000000,16
0,70,0.999999,48
0,96,0.999996,5
0,65,0.999996,51
0,22,0.999996,3
0,59,0.999995,54
0,82,0.999995,97
0,63,0.999991,36
0,88,0.999990,56
0,65,0.999985,36


In [16]:
data(result.sort_values('blz', ascending = False).head(100))

-------------------------------------
Количество данных - 100
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - df1 || Кол-во уникальных значений - 20 || Кол-во дубликатов - 80

Уникальные значения:
[13 22 27 29 34 59 60 61 62 63 65 68 70 71 76 82 84 88 93 96]

-------------------------------------
 Столбец - blz || Кол-во уникальных значений - 100 || Кол-во дубликатов - 0

Уникальные значения:
[0.99973008 0.99973144 0.99973206 0.99973966 0.99974499 0.99974704
 0.99975114 0.99976717 0.99976995 0.999772   0.99977502 0.99977577
 0.99978151 0.99978261 0.99978317 0.99978404 0.99978799 0.99978832
 0.99978903 0.99979584 0.99979739 0.99979806 0.99979813 0.99980584
 0.99980746 0.99981147 0.99981201 0.9998126  0.99982562 0.99982587
 0.99982927 0.99982973 0.99983142 0.99983417 0.99983505 0.99983544
 0.99983661 0.99984085 0.99984119 0.99985391 0.99985439 0.99985735
 0.99985823 0.99985864 0.99985933 0.99985937 0.99986077 0.99986239
 0

In [17]:
res100 = result.sort_values('blz', ascending = False).head(100)

In [18]:
for a in res100['df1']:
    print('______________________\n', res100[res100['df1'] == a])

______________________
   df1       blz df2
0  61  1.000000  16
0  61  0.999921   3
0  61  0.999751  44
______________________
   df1       blz df2
0  70  0.999999  48
0  70  0.999961  64
0  70  0.999919  74
0  70  0.999893  51
0  70  0.999880   5
0  70  0.999866  46
0  70  0.999798  36
0  70  0.999772  80
0  70  0.999770  66
______________________
   df1       blz df2
0  96  0.999996   5
0  96  0.999861  48
______________________
   df1       blz df2
0  65  0.999996  51
0  65  0.999985  36
0  65  0.999971  64
0  65  0.999967  74
0  65  0.999858   1
0  65  0.999854  48
0  65  0.999830  80
0  65  0.999807  66
0  65  0.999784  46
0  65  0.999731  56
______________________
   df1       blz df2
0  22  0.999996   3
0  22  0.999925  16
______________________
   df1       blz df2
0  59  0.999995  54
0  59  0.999980  37
0  59  0.999978  44
0  59  0.999937  26
0  59  0.999806  31
0  59  0.999788  46
______________________
   df1       blz df2
0  82  0.999995  97
______________________
   df1   

In [19]:
def srav(data, a, b):
    print(df.loc[[a]])
    print('\n', data[(data['df1'] == a) & (data['df2'] == b)], '\n')
    print(df.loc[[b]])

In [20]:
srav(result, 61, 16)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
61    537235            6        24        44.98                  3

   df1  blz df2
0  61  1.0  16 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
16    131111            6        17       45.018                  3


In [21]:
srav(result, 61, 3)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
61    537235            6        24        44.98                  3

   df1       blz df2
0  61  0.999921   3 

   order_gk  weekday_key  hour_key  distance_km  offer_class_group
3     37826            5        19       36.971                  2


In [22]:
srav(result, 61, 44)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
61    537235            6        24        44.98                  3

   df1       blz df2
0  61  0.999751  44 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
44    336264            6        23        53.07                  3


In [23]:
srav(result, 70, 48)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
70    607437            3        25        35.47                  1

   df1       blz df2
0  70  0.999999  48 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
48    402616            3        19        35.02                  1


In [24]:
srav(result, 70, 64)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
70    607437            3        25        35.47                  1

   df1       blz df2
0  70  0.999961  64 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
64    554260            7        20       90.751                  3


In [25]:
srav(result, 70, 74)

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
70    607437            3        25        35.47                  1

   df1       blz df2
0  70  0.999919  74 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
74    635571            2        19       25.977                  1


---

Как мы можем видеть находится весьма много схожих данных и при более детальном рассмотрении они не всегда являются дубликатами.

Попробуем для каждой поездки с ошибочным временем найти лучшее совпадение.

---

In [26]:
result_1 = pd.DataFrame(columns=['df1', 'blz', 'df2'])

for ind1 in tqdm(df1.index):
    blz = 0.8
    ind = 777
    
    for ind2 in df2.index:
        cs = cosine_similarity (df1.loc[[ind1]], df2.loc[[ind2]])
        if cs[0][0] > blz:
            blz = cs[0][0]
            ind = ind2
 
        
    res = pd.DataFrame({'df1': [ind1],
                        'blz': [blz],
                        'df2': [ind]})
    result_1 = pd.concat([result_1, res])
                            
result_1

  0%|          | 0/32 [00:00<?, ?it/s]

,df1,blz,df2
0,0,0.999685,41
0,9,0.999166,97
0,13,0.999886,5
0,22,0.999996,3
0,25,0.958611,35
0,27,0.999924,48
0,29,0.999826,21
0,30,0.993854,14
0,34,0.999982,6
0,45,0.999672,86


In [27]:
data(result_1)

-------------------------------------
Количество данных - 32
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - df1 || Кол-во уникальных значений - 32 || Кол-во дубликатов - 0

Уникальные значения:
[0 9 13 22 25 27 29 30 34 45 58 59 60 61 62 63 65 67 68 70 71 76 78 81 82
 84 88 90 92 93 96 99]

-------------------------------------
 Столбец - blz || Кол-во уникальных значений - 32 || Кол-во дубликатов - 0

Уникальные значения:
[0.9586109  0.99385399 0.99655753 0.99710191 0.99892045 0.99916637
 0.99948242 0.99952919 0.99957851 0.99961078 0.99967173 0.99968539
 0.99982562 0.99983142 0.99985933 0.9998859  0.9998976  0.99992365
 0.99993498 0.99994141 0.99996325 0.99996857 0.99998212 0.99999048
 0.99999076 0.99999471 0.99999497 0.99999568 0.99999617 0.99999642
 0.99999935 0.99999999]

-------------------------------------
 Столбец - df2 || Кол-во уникальных значений - 22 || Кол-во дубликатов - 10

Уникальные значения:
[3 5 6 11 

In [28]:
result_1.sort_values('blz', ascending = False).head(7)

,df1,blz,df2
0,61,1.000000,16
0,70,0.999999,48
0,96,0.999996,5
0,65,0.999996,51
0,22,0.999996,3
0,59,0.999995,54
0,82,0.999995,97


In [29]:
for a, b in zip(result_1.sort_values('blz', ascending = False).head(7)['df1'], 
                result_1.sort_values('blz', ascending = False).head(7)['df2']):
    srav(result_1, a, b)
    print('\n______________________________________________________________________\n')

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
61    537235            6        24        44.98                  3

   df1  blz df2
0  61  1.0  16 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
16    131111            6        17       45.018                  3

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
70    607437            3        25        35.47                  1

   df1       blz df2
0  70  0.999999  48 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
48    402616            3        19        35.02                  1

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
96    838313            5        24       49.928                  1

   df1       blz df2
0  96  0.999996   5 

   order_gk  weekday_key  hour_key  distance_km  off

---

Из полученных результатов так же видно, что лучшее совпадение тоже дает не совсем хорошие результаты. 

Имеются дубликаты по подобранным парам. 

Если рассматривать подобранные пары более подробно, можно увидеть, что они явно не являются дубликатами.

Теперь попробуем найти лучшие совпадения с учетом часа поездки.

---

In [30]:
df3 = df.copy()
df3.drop('order_gk', axis= 1 , inplace= True )
df3

,weekday_key,hour_key,distance_km,offer_class_group
0,3,27,14.239,1
1,3,21,39.518,2
2,2,21,17.203,2
3,5,19,36.971,2
4,2,19,49.634,2
...,...,...,...,...
95,4,22,11.877,3
96,5,24,49.928,1
97,2,15,10.893,3
98,4,20,29.096,3


In [31]:
result_2 = pd.DataFrame(columns=['df1', 'blz', 'df2'])
sp = set(df3.index)

for ind1 in tqdm(df3.index):
    sp.remove(ind1)
    for ind2 in sp:
        cs = cosine_similarity (df3.loc[[ind1]], df3.loc[[ind2]])

        res = pd.DataFrame({'df1': [ind1],
                            'blz': [cs[0][0]],
                            'df2': [ind2]})
        result_2 = pd.concat([result_2, res])
                            
result_2

  0%|          | 0/100 [00:00<?, ?it/s]

,df1,blz,df2
0,0,0.827928,1
0,0,0.978949,2
0,0,0.821306,3
0,0,0.751711,4
0,0,0.758175,5
...,...,...,...
0,96,0.985860,98
0,96,0.921482,99
0,97,0.941354,98
0,97,0.990731,99


In [32]:
result_2.sort_values('blz', ascending = False).head(20)

,df1,blz,df2
0,10,1.000000,43
0,68,0.999959,80
0,44,0.999922,59
0,13,0.999901,46
0,43,0.999852,53
0,10,0.999851,53
0,37,0.999830,66
0,3,0.999823,61
0,36,0.999804,48
0,70,0.999791,74


In [33]:
data(result_2.sort_values('blz', ascending = False).head(100))

-------------------------------------
Количество данных - 100
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - df1 || Кол-во уникальных значений - 48 || Кол-во дубликатов - 52

Уникальные значения:
[1 2 3 4 5 6 8 10 11 12 13 15 20 21 22 23 24 26 27 29 31 32 36 37 38 41 43
 44 46 49 51 54 56 57 59 60 62 63 65 66 68 69 70 71 72 74 79 89]

-------------------------------------
 Столбец - blz || Кол-во уникальных значений - 100 || Кол-во дубликатов - 0

Уникальные значения:
[0.99880663 0.99883336 0.9988371  0.99884208 0.99884498 0.99884776
 0.99886376 0.9988678  0.9988757  0.99888281 0.99889142 0.99890739
 0.99891105 0.99892154 0.99892289 0.99894519 0.99894697 0.99896787
 0.9989723  0.99899845 0.99899935 0.99902028 0.99902437 0.99903377
 0.99903472 0.99904042 0.9990422  0.99907212 0.99909556 0.99910075
 0.99910281 0.99910418 0.99911071 0.99911301 0.99911786 0.99913377
 0.99915219 0.99915637 0.99915962 0.99917517 0.99922087 0.

In [34]:
result_2.sort_values('blz', ascending = False).head(7)

,df1,blz,df2
0,10,1.000000,43
0,68,0.999959,80
0,44,0.999922,59
0,13,0.999901,46
0,43,0.999852,53
0,10,0.999851,53
0,37,0.999830,66


In [35]:
for a, b in zip(result_2.sort_values('blz', ascending = False).head(7)['df1'], 
                result_2.sort_values('blz', ascending = False).head(7)['df2']):
    srav(result_2, a, b)
    print('\n______________________________________________________________________\n')

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
10     94794            2        20       57.879                  3

   df1  blz df2
0  10  1.0  43 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
43    330292            2        20       57.846                  3

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
68    594666            5        30       81.507                  2

   df1       blz df2
0  68  0.999959  80 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
80    693398            4        22       60.169                  1

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
44    336264            6        23        53.07                  3

   df1       blz df2
0  44  0.999922  59 

    order_gk  weekday_key  hour_key  distance_km  of

---

Лучшие совпадения с учетом часа поездки также не дало хороших результатов.

При более детальном рассмотрении подобранных пар мы видим, что даже лучшие результаты имеют серьезные различия как по классу такси, так и по дистанции поездки.

Попробуем найти ближайших соседей методом NearestNeighbors.

---

In [36]:
from sklearn.neighbors import NearestNeighbors

In [37]:
neigh = NearestNeighbors(n_neighbors=5, metric='euclidean')
neigh.fit(df3)

NearestNeighbors(metric='euclidean')

In [38]:
result_3 = pd.DataFrame(columns=['df1', 'blz', 'df2'])

for ind1 in tqdm(df3.index):
    if not ind1 in result_3['df2'].values:
        res = neigh.kneighbors(df3.loc[[ind1]], return_distance=True)
        result_3 = pd.concat(
                        [result_3,
                         pd.DataFrame({
                              'df1': [ind1],
                              'blz': [res[0][0][1]],
                              'df2': [df3.iloc[res[1][0][1]].name]})])
                           
result_3

  0%|          | 0/100 [00:00<?, ?it/s]

,df1,blz,df2
0,0,1.684950,84
0,1,2.223966,56
0,2,2.214411,57
0,3,2.967558,48
0,4,3.028431,72
...,...,...,...
0,92,3.000620,63
0,93,2.805190,84
0,94,2.957064,31
0,96,3.983988,44


In [39]:
result_3.sort_values('blz').head(20)

,df1,blz,df2
0,10,0.033000,43
0,49,1.092923,69
0,24,1.441472,40
0,9,1.448000,82
0,79,1.504000,98
0,37,1.585989,66
0,0,1.684950,84
0,20,1.890110,40
0,8,1.992087,41
0,35,2.005019,42


In [40]:
data(result_3)

-------------------------------------
Количество данных - 65
-------------------------------------
Количество дубликатов - 0

-------------------------------------
 Столбец - df1 || Кол-во уникальных значений - 65 || Кол-во дубликатов - 0

Уникальные значения:
[0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 24 25 26 27 28
 30 31 32 33 34 35 37 38 44 45 46 49 50 54 55 58 62 63 64 68 73 74 75 76
 79 81 83 86 88 89 90 91 92 93 94 96 97]

-------------------------------------
 Столбец - blz || Кол-во уникальных значений - 65 || Кол-во дубликатов - 0

Уникальные значения:
[ 0.033       1.09292314  1.44147182  1.448       1.504       1.58598897
  1.68494985  1.89011005  1.99208659  2.0050187   2.03695459  2.03980416
  2.21441098  2.22396605  2.26965218  2.27703755  2.31157868  2.36168076
  2.39862961  2.45002061  2.45401732  2.48147718  2.528178    2.56291163
  2.5925555   2.675       2.69799203  2.79110086  2.80518965  2.91611317
  2.92241612  2.93233849  2.94857321  2.95706358 

In [41]:
result_3.sort_values('blz').head(7)

,df1,blz,df2
0,10,0.033000,43
0,49,1.092923,69
0,24,1.441472,40
0,9,1.448000,82
0,79,1.504000,98
0,37,1.585989,66
0,0,1.684950,84


In [42]:
for a, b in zip(result_3.sort_values('blz').head(7)['df1'], 
                result_3.sort_values('blz').head(7)['df2']):
    srav(result_3, a, b)
    print('\n______________________________________________________________________\n')

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
10     94794            2        20       57.879                  3

   df1    blz df2
0  10  0.033  43 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
43    330292            2        20       57.846                  3

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
49    412671            3        21       22.122                  3

   df1       blz df2
0  49  1.092923  69 

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
69    597863            3        20       22.563                  3

______________________________________________________________________

    order_gk  weekday_key  hour_key  distance_km  offer_class_group
24    190596            6        22       11.348                  2

   df1       blz df2
0  24  1.441472  40 

    order_gk  weekday_key  hour_key  distance_km

In [43]:
result.sort_values('blz', ascending = False).head(7)

,df1,blz,df2
0,61,1.000000,16
0,70,0.999999,48
0,96,0.999996,5
0,65,0.999996,51
0,22,0.999996,3
0,59,0.999995,54
0,82,0.999995,97


In [44]:
result_1.sort_values('blz', ascending = False).head(7)

,df1,blz,df2
0,61,1.000000,16
0,70,0.999999,48
0,96,0.999996,5
0,65,0.999996,51
0,22,0.999996,3
0,59,0.999995,54
0,82,0.999995,97


In [45]:
result_2.sort_values('blz', ascending = False).head(7)

,df1,blz,df2
0,10,1.000000,43
0,68,0.999959,80
0,44,0.999922,59
0,13,0.999901,46
0,43,0.999852,53
0,10,0.999851,53
0,37,0.999830,66


In [46]:
result_3.sort_values('blz').head(7)

,df1,blz,df2
0,10,0.033000,43
0,49,1.092923,69
0,24,1.441472,40
0,9,1.448000,82
0,79,1.504000,98
0,37,1.585989,66
0,0,1.684950,84


In [47]:
result_3[result_3['df2'].duplicated()]

,df1,blz,df2
0,16,4.653687,72
0,24,1.441472,40
0,25,5.041674,39
0,31,2.592555,70
0,34,4.483413,29
0,38,8.570892,51
0,46,2.450021,69
0,49,1.092923,69
0,54,2.993343,2
0,68,8.280097,51


In [48]:
for a in set(result_3[result_3['df2'].duplicated()]['df2']):
    print(result_3[result_3['df2'] == a], '\n__________________________________________________\n')

  df1       blz df2
0  33  3.459517   2
0  54  2.993343   2 
__________________________________________________

  df1       blz df2
0  18  2.916113  69
0  46  2.450021  69
0  49  1.092923  69 
__________________________________________________

  df1       blz df2
0  22  2.562912  70
0  31  2.592555  70
0  88  3.286834  70 
__________________________________________________

  df1       blz df2
0  19  3.713654  39
0  25  5.041674  39 
__________________________________________________

  df1       blz df2
0   4  3.028431  72
0  16  4.653687  72 
__________________________________________________

  df1       blz df2
0  20  1.890110  40
0  24  1.441472  40 
__________________________________________________

  df1       blz df2
0  14  2.454017  47
0  73  2.398630  47 
__________________________________________________

  df1       blz df2
0  21  5.362651  51
0  38  8.570892  51
0  68  8.280097  51 
__________________________________________________

  df1       blz df2
0   0  1.684950 

---

### Вывод:

Для анализа задублированных поездок в датасете над данными были проделаны различные методы и применены различные гипотезы. 
После каждого проделанного метода или применённой гипотезы мы получали хорошие статистические данные, которые на первый взгляд нас полностью устраивают, и мы могли бы сказать – вот оно за двоение! 
Но, каждый раз разбирая более детально полученные результаты мы видим те или иные различия в разных столбцах, то дистанция не та, то класс или день недели не совпадали.

Из предоставленных данных и проделанных тестов можно сделать вывод, что за двоение поездок не было, а был сбой в правильности интерпретации часа поездки.

---